In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the datasets

In [ ]:
df_train=pd.read_csv('/kaggle/input/titanic/train.csv')
df_test=pd.read_csv('/kaggle/input/titanic/test.csv')

# EDA

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

In [ ]:
df_test.head()

In [ ]:
df_test.describe()

In [ ]:
df_test.info()

# Data Cleaning

In [ ]:
train_mean_age=df_train['Age'].mean()

In [ ]:
train_mean_age

In [ ]:
df_train['Age']=df_train['Age'].fillna(train_mean_age)

In [ ]:
df_train.info()

In [ ]:
test_mean_age=df_test['Age'].mean()

In [ ]:
test_mean_age

In [ ]:
df_test['Age']=df_test['Age'].fillna(test_mean_age)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
lab= LabelEncoder()

In [ ]:
df_train['Sex']=lab.fit_transform(df_train['Sex'])
df_test['Sex']=lab.fit_transform(df_test['Sex'])



In [ ]:
df_train=df_train[~df_train['Embarked'].isnull()]
df_test=df_test[~df_test['Embarked'].isnull()]

In [ ]:
df_train['Embarked']=lab.fit_transform(df_train['Embarked'])
df_test['Embarked']=lab.fit_transform(df_test['Embarked'])

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_test['Fare'].isnull().any()
df_test['Fare'].fillna(0, inplace=True)
df_test.info()


# Model Fitting

In [ ]:
features=['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']

In [ ]:
X=df_train[features]
y=df_train.Survived

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.33, random_state=42)
X_train.head()


In [ ]:
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.fit_transform(X_test)


In [ ]:

from sklearn.linear_model import LogisticRegression


logreg = LogisticRegression()


logreg.fit(rescaledX_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix



y_pred = logreg.predict(X_test)

print("Accuracy of logistic regression classifier: ", logreg.score(rescaledX_test,y_test))


print(confusion_matrix(y_test,y_pred))

# Grid-Search CV

In [ ]:
from sklearn.model_selection import GridSearchCV


tol = [0.01,0.001,0.0001]
max_iter = [100,150,200]

param_grid = dict(tol=tol,max_iter=max_iter)

In [ ]:

grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)


rescaledX = scaler.fit_transform(X)


grid_model_result = grid_model.fit(rescaledX, y)


print("Best: %f using %s" % (grid_model.best_score_, grid_model.best_params_))



In [ ]:
final_model=LogisticRegression(max_iter=100, tol=0.01)
final_model.fit(X,y)
final_pred=final_model.predict(df_test[features])
final_pred

#  SAVING


In [ ]:
output= pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': final_pred})
output.to_csv('MySubmission.csv', index=False)
print('Success')

# New Model